In [35]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import warnings
from astropy.wcs import WCS
from astropy.io import fits
from scipy.optimize import curve_fit, root_scalar
from spectral_cube import SpectralCube
from lmfit import Model, Parameters
from scipy.interpolate import interp1d
import aplpy
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.mixture import GaussianMixture

warnings.filterwarnings('ignore')


In [2]:
# Constants
etamb = 0.89
etaf = 1.0
nu11 = 23.6944955e9
nu22 = 23.7226333e9
h = 6.62606896e-34
kB = 1.3806504e-23
Tbg = 2.73
c = 2.99792458e8
To = 41.5
epsilon = 8.854187817e-12
dipole = 298117.06e6
C_moment = 186726.36e6
mu = 1.476 * 3.336e-30
mu11 = mu ** 2 * (1. / 2.)
Einstein_A = (16. * np.pi ** 3 / (3. * epsilon * h * c ** 3)) * nu11 ** 3 * abs(mu11)
J_Tbg = h * nu11 / kB * (1. / (np.exp(h * nu11 / (kB * Tbg)) - 1))
etac = etamb * etaf
mNH3=17.03      #amu
mH=1.00794      #amu
amu=1.66053886e-27  #kg

In [70]:
def gaussian(x: np.ndarray, amp: float, cen: float, wid: float) -> np.ndarray:
    return amp * np.exp(-((x - cen)**2) / (2 * wid**2))

def quadrupole(x: np.ndarray, 
               m_amp: float, m_vel: float, m_wid: float, 
               s1_amp: float, s2_amp: float, s3_amp: float, s4_amp: float) -> np.ndarray:
    vel = [-26.023, -16.368, 0.0, 16.368, 26.023]
    return (gaussian(x, s1_amp, m_vel + vel[0], m_wid) +
            gaussian(x, s2_amp, m_vel + vel[1], m_wid) +
            gaussian(x, m_amp, m_vel , m_wid) +
            gaussian(x, s3_amp, m_vel + vel[3], m_wid) +
            gaussian(x, s4_amp, m_vel + vel[4], m_wid))
    
def quad_curvefit(xdata: np.ndarray, ydata: np.ndarray):
    amp_guesses = np.array([0.063, 0.039, 1.0, 0.092, 0.063])*np.max(ydata)
    mvel_guess = xdata[np.argmax(ydata)]
    mwid_guess = 1
    init_guesses = np.array([amp_guesses[2], mvel_guess, mwid_guess, 
                             amp_guesses[0], amp_guesses[1], 
                             amp_guesses[3], amp_guesses[4]])
    bounds = ([0, -np.inf, 0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf])
    popt, pcov = curve_fit(quadrupole, xdata, ydata, p0=init_guesses, bounds=bounds)
    return popt, pcov

def solve_Tk(Tr): # Solve for kinetic temperature Tk given rotational temperature Tr
    if np.isnan(Tr):
        return np.nan

    def KineticTemp(Tk):
        if Tk <= 0:
            return 1e6  # prevent log or exp issues
        return Tk / (1. + ((Tk / To) * np.log(1. + (0.6 * np.exp(-15.7 / Tk))))) - Tr

    try:
        sol = root_scalar(KineticTemp, bracket=[1, 500], method='brentq')
        return sol.root if sol.converged else np.nan
    except Exception:
        return np.nan

In [ ]:
datapath = '/home/scratch/hfwest/RAMPS-Data/'
resultpath = '/home/scratch/hfwest/RAMPS-Results/'

Cube11 = SpectralCube.read(datapath + 'Data/Pilot_NH3_11_bl2.fits')
Cube22 = SpectralCube.read(datapath + 'Data/Pilot_NH3_22_bl2.fits')

# Grab the middle channel as a 2D slice
slice2d11 = Cube11[Cube11.spectral_axis.size // 2]
slice2d22 = Cube22[Cube22.spectral_axis.size // 2]

# Get the WCS from the 2D slice
wcs_3d11 = Cube11.wcs
wcs_3d22 = Cube22.wcs
wcs_2d11 = slice2d11.wcs
wcs_2d22 = slice2d22.wcs

# Convert to header
header_3d11 = wcs_3d11.to_header()
header_3d22 = wcs_3d22.to_header()
header_2d11 = wcs_2d11.to_header()
header_2d22 = wcs_2d22.to_header()

vmargin = 50
threshold = 5
thresholdlower = 3
vl = 140 * u.km / u.s # Adjust for specific maps
vh = 260 * u.km / u.s # Adjust for specific maps
rmsvl = 280 * u.km / u.s # Adjust for specific maps
rmsvh = 320 * u.km / u.s # Adjust for specific maps

In [ ]:
Cube11 = Cube11.with_spectral_unit(u.km/u.s, velocity_convention='radio', rest_value=nu11*u.Hz)
Cube22 = Cube22.with_spectral_unit(u.km/u.s, velocity_convention='radio', rest_value=nu22*u.Hz)

Cube11_slab = Cube11.spectral_slab(vl, vh)
RMS11_slab = Cube11.spectral_slab(rmsvl, rmsvh)
vel_axis11 = Cube11_slab.spectral_axis
cube11_data = Cube11_slab.unmasked_data[:].value
rms11_data = RMS11_slab.unmasked_data[:].value

Cube22_slab = Cube22.spectral_slab(vl, vh)
RMS22_slab = Cube22.spectral_slab(rmsvl, rmsvh)
vel_axis22 = Cube22_slab.spectral_axis
cube22_data = Cube22_slab.unmasked_data[:].value
rms22_data = RMS22_slab.unmasked_data[:].value

ny, nx = Cube11.shape[1], Cube11.shape[2]
nz11 = vel_axis11.shape[0]
nz22 = vel_axis22.shape[0]

TMaxMap11 = np.full((ny, nx), np.nan)
TMaxMap22 = np.full((ny, nx), np.nan)
TauMap11 = np.full((ny, nx), np.nan)
TauMap22 = np.full((ny, nx), np.nan)
SigmaMap11 = np.full((ny, nx), np.nan)
SigmaMap22 = np.full((ny, nx), np.nan)
VelMap11 = np.full((ny, nx), np.nan)
VelMap22 = np.full((ny, nx), np.nan)
TempRotMap = np.full((ny, nx), np.nan)
TempExMap = np.full((ny, nx), np.nan)
TempKinMap = np.full((ny, nx), np.nan)
FFMap = np.full((ny, nx), np.nan)
NColMap = np.full((ny, nx), np.nan)
Sigma_Therm_Map = np.full((ny, nx), np.nan)
Sigma_Turb_Map11 = np.full((ny, nx), np.nan)
Sigma_Turb_Map22 = np.full((ny, nx), np.nan)
TMaxMap11Comp1 = np.full((ny, nx), np.nan)
TMaxMap11Comp2 = np.full((ny, nx), np.nan)
SigmaMap11Comp1 = np.full((ny, nx), np.nan)
SigmaMap11Comp2 = np.full((ny, nx), np.nan)
VelMap11Comp1 = np.full((ny, nx), np.nan)
VelMap11Comp2 = np.full((ny, nx), np.nan)
TMaxMap22Comp1 = np.full((ny, nx), np.nan)
TMaxMap22Comp2 = np.full((ny, nx), np.nan)
SigmaMap22Comp1 = np.full((ny, nx), np.nan)
SigmaMap22Comp2 = np.full((ny, nx), np.nan)
VelMap22Comp1 = np.full((ny, nx), np.nan)
VelMap22Comp2 = np.full((ny, nx), np.nan)
ResidMap11 = np.full((ny, nx), np.nan)
ResidMap22 = np.full((ny, nx), np.nan)
ResidCube11 = np.full((nz11, ny, nx), np.nan)
ResidCube22 = np.full((nz22, ny, nx), np.nan)
RedChiMap11 = np.full((ny, nx), np.nan)
RedChiMap22 = np.full((ny, nx), np.nan)

In [ ]:
for j in range(ny):
    for i in range(nx):
        spec11 = cube11_data[:, j, i] # Extract (1,1) spectrum
        rms11 = np.std(rms11_data[:, j, i])  # Extract RMS baseline noise level
        testmaxindex11 = np.argmax(spec11)          # Index of velocity corresponding to largest peak in (1,1) spectrum
        testvel_peak11 = vel_axis11[testmaxindex11] # Velocity corresponding to largest peak in (1,1) spectrum
        amp011 = spec11[testmaxindex11]             # Height of largest peak in (1,1) spectrum

        # Repeat for (2,2) spectrum
        spec22 = cube22_data[:, j, i]
        rms22 = np.std(rms22_data[:, j, i])
        testmaxindex22 = np.argmax(spec22)
        testvel_peak22 = vel_axis22[testmaxindex22]
        amp022 = spec22[testmaxindex22]

        if amp011 > threshold * rms11: # If peak of (1,1) spectrum is greater than 5 sigma above the baseline, then we fit to it
            mask11 = (vel_axis11.value > testvel_peak11.value - vmargin) & (vel_axis11.value < testvel_peak11.value + vmargin)
            # Value mask: velocities within |vmargin| distance of the peak velocity. In this case, vmargin = 50 km/s
            vel_axis11_clip = vel_axis11[mask11]
            spec11_clip = spec11[mask11]

            fit_popt, fit_pcov = quad_curvefit(vel_axis11_clip.value, spec11_clip)
            
            # TODO fill in best fit parameters
            Tau11_main = # best fit tau value from main/sattelite ratio
            TMax11 = # best fit main amplitude from overall signal peak
            SigmaMap11[j,i] = # best fit velocity dispersion
            VelMap11[j,i] = # best fit main line velocity
            TMaxMap11Comp1[j,i] = # best fit main amplitude from overall signal peak
            SigmaMap11Comp1[j,i] = # best fit velocity dispersion
            VelMap11Comp1[j,i] = # best fit main line velocity
            
            # TODO Residuals and red chi-squared
            InterpResid11 = interp1d(vel_axis11_clip, resultq11.residual, bounds_error=False, fill_value=0.0)
            ResidCube11[:,j,i] = InterpResid11(vel_axis11)
            ResidMap11[j,i] = np.sqrt(np.mean(resultq11.residual**2))
            RedChiMap11[j,i] = resultq11.redchi
            
            TauMap11[j,i] = Tau11_main * 1.995 # (1,1 quadrupoles are 0.226, 0.273, 0.277, 0.219, sum = 0.995 of the main height, so the total is 1.995 and the ratio of the main line to the total is 1/1.995)
            TMaxMap11[j,i] = TMax11
            

        if amp022 > thresholdlower * rms22: # If peak of (2,2) spectrum is greater than 3 sigma above the baseline
            mask22 = (vel_axis22.value > testvel_peak22.value - vmargin) & (vel_axis22.value < testvel_peak22.value + vmargin)
            # Value mask: velocities within |vmargin| distance of the peak velocity. In this case, vmargin = 50 km/s
            vel_axis22_clip = vel_axis22[mask22]
            spec22_clip = spec22[mask22]

            fit_popt, fit_pcov = quad_curvefit(vel_axis22_clip.value, spec22_clip)
            
            # TODO fill in best fit parameters
            Tau22_main = # best fit tau value from main/sattelite ratio
            TMax22 = # best fit main amplitude from overall signal peak
            SigmaMap22[j,i] = # best fit velocity dispersion
            VelMap22[j,i] = # best fit main line velocity
            TMaxMap22Comp1[j,i] = # best fit main amplitude from overall signal peak
            SigmaMap22Comp1[j,i] = # best fit velocity dispersion
            VelMap22Comp1[j,i] = # best fit main line velocity
            
            # TODO Residuals and red chi-squared
            InterpResid22 = interp1d(vel_axis22_clip, resultq22.residual, bounds_error=False, fill_value=0.0)
            ResidCube22[:,j,i] = InterpResid22(vel_axis11)
            ResidMap22[j,i] = np.sqrt(np.mean(resultq22.residual**2))
            RedChiMap22[j,i] = resultq22.redchi
            
            TauMap22[j,i] = Tau22_main * 1.995 # (1,1 quadrupoles are 0.226, 0.273, 0.277, 0.219, sum = 0.995 of the main height, so the total is 1.995 and the ratio of the main line to the total is 1/1.995)
            TMaxMap22[j,i] = TMax22

        # TODO Check everything past here
        Tr = (-1.*To)/(np.log((-0.282/TauMap11[j,i]/1.995)*np.log(1.-((TMaxMap22[j,i]/TMaxMap11[j,i])*(1.-np.exp(-1.*TauMap11[j,i]/1.995))))))
        TempRotMap[j,i] = Tr

        Tk = solve_Tk(Tr)
        TempKinMap[j,i] = Tk
        B=1.-np.exp(-1.*TauMap11[j,i]/1.995)
        TempExMap[j,i] = h*nu11/(kB*np.log((h*nu11/kB)*(1./(TMaxMap11[j,i]/(etac*B)+J_Tbg)+1)))
#Calculate filling factor by setting Tex=Tkin
        J_Tkin=h*nu11/kB*(1./((np.exp(h*nu11/(kB*Tk)))-1))
        FFMap[j,i]=TMaxMap11[j,i]/(etamb*(J_Tkin-J_Tbg)*B)
        Sigma_Therm_Map[j,i]=((8.*kB*Tk*np.log(2.)/(mNH3*amu))**0.5)/1000.
        Sigma_Turb_Map11[j,i]=((SigmaMap11[j,i]**2.)-(Sigma_Therm_Map[j,i]**2.))**0.5
        Sigma_Turb_Map22[j,i]=((SigmaMap22[j,i]**2.)-(Sigma_Therm_Map[j,i]**2.))**0.5

#Calculating column density - using sigma, not dV
        N11=(8.*np.pi*(nu11**2.)/c**2.)*(1./Einstein_A)*((1.+np.exp(-1.*(h*nu11/(kB*Tk))))/(1.-np.exp(-1.*(h*nu11/(kB*Tk)))))*(np.sqrt(2.*np.pi))*(np.sqrt(((SigmaMap11[j,i]*1e3)**2)/(8.*np.log(2))))*(nu11/c)*TauMap11[j,i]/1e4

        # No. of states to sum over for partition function, 10 is more than sufficient
        Z_tot=0.
        Nstates=10
        for jstate in range(1,Nstates):
        # ortho=0,3,6,9...
        # if (jstate mod 3) ne 0 then state is para- (J=1,2,4,5...)
            if (jstate % 3) != 0:
                S=1
            else:
                S=0
            Z=(2.*jstate+1.)*S*np.exp(-1.*(h/(kB*Tk))*((dipole*jstate*(jstate+1))+((C_moment-dipole)*jstate**2)))
            Z_tot=Z_tot+Z
        Z_11=3.*np.exp(-1.*h*(dipole+C_moment)/(kB*Tk))
        NColMap[j,i] = np.log(N11*Z_tot/Z_11) # Only 1 column density map created, corresponding only to (1,1) line

In [ ]:
# TODO save results in fits